## Based on the followings:
* http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
* http://adventuresinmachinelearning.com/keras-lstm-tutorial/
* https://machinelearningmastery.com/configure-encoder-decoder-model-neural-machine-translation/
* https://machinelearningmastery.com/develop-encoder-decoder-model-sequence-sequence-prediction-keras/
* https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
* https://github.com/farizrahman4u/seq2seq
* https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

## TODO
* ~~look into categorical representation~~
* ~~look into the number of missing words over the total~~
* ~~look into different models (attention, hierachical, etc.)~~
* look into attention

In [13]:
import os
import re
import numpy as np
import glob
import pickle as pkl
import tensorflow as tf
import keras.backend as K
import matplotlib.pyplot as plt
import time

from string import punctuation
from itertools import islice
from gensim.models import KeyedVectors
from keras.models import Model
from keras.layers import Layer, Input, LSTM, GRU, Dense, Masking, Embedding, Activation, TimeDistributed
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, Callback

from sklearn.model_selection import KFold

In [2]:
DATA_PATH = 'data'
OUTPUT_PATH = 'output'
punct = set(punctuation)
file_list = sorted(glob.glob('data/parsed/*.txt'))

In [6]:
w2v = KeyedVectors.load_word2vec_format(os.path.join(DATA_PATH, 'GoogleNews-vectors-negative300.bin.gz'), binary=True)

In [3]:
# re-load params
with open(os.path.join(DATA_PATH, 'data_20k.pkl'), 'rb') as data_file, open(os.path.join(DATA_PATH, 'params_20k.pkl'), 'rb') as params_file:
    data = pkl.load(data_file)
    params = pkl.load(params_file)
    tokenizer = params['tokenizer']
    index_word = params['index_word']
    word2embeddings = params['w2e']
    embedding_matrix = params['W']
    missing_words = params['missing_words']

In [4]:
embedding_dim = 300#w2v.vector_size
eos_token = 'EOS'
unk_token = 'UNK'
eos_vector = np.ones((embedding_dim))
unk_vector = np.zeros((embedding_dim))

In [5]:
def preprocess(text):
    text = re.sub(repl='', string=text, pattern='^> ') # remove starting caret, if any
    text = re.sub(repl='\g<1> \g<2>', string=text, pattern='(\w+)-(\w+)') # compound words    
    text = re.sub(repl=' ', string=text, pattern='-{2,}|\s{2,}|[%s\t\n/]' % (''.join(punctuation)))
#     text = re.sub(repl=' digits ', string=text, pattern='^\d+$| \d+| \d+ ') # replace digits with a standard 'digits' word
    return text

def read_corpus(file_list):
    corpus = []
    for file in file_list:
        with open(file, 'r', encoding='utf-8') as f:
            print('read_corpus: processing [{}]'.format(file))
            corpus.append(f.read())
            
    return corpus
            
def build_vocabulary(corpus, num_words, oov_token):
    tokenizer = Tokenizer(num_words=num_words+1, oov_token=oov_token) # +1 for the oov token
    tokenizer.fit_on_texts(corpus)
    
    # Fix keras' nasty behaviour. See https://github.com/keras-team/keras/issues/8092
    # Only include words found in w2v
    tokenizer.word_index = {w:i for  w,i in tokenizer.word_index.items() 
                                if   i <= num_words} # <= because tokenizer is 1 indexed (this will leave out UNK)
    tokenizer.num_words = num_words + 2  # UNK + EOS
    tokenizer.word_index[oov_token] = len(tokenizer.word_index) + 1
    tokenizer.word_index[eos_token] = len(tokenizer.word_index) + 1
    index_word = [None for i in range(len(tokenizer.word_index) + 1)]  # index is 1-based
    for w,i in tokenizer.word_index.items():
        index_word[i] = w
    
    return tokenizer, index_word

def prepare_data(corpus, tokenizer):
    # Still go through the files line by line, as we want to predict the next scene, 
    # not just the next sentence
    data = []
    for i, doc in enumerate(corpus):
        doc_data = []
        print('prepare_data: processing [{}]'.format(file_list[i]))
        
        for j, line in enumerate(doc.split('\n')):
            if len(line) == 0:
                print('Line {} is empty. Replacing with "empty line".'.format(j+1))
                line = 'empty line'

            doc_data.append(tokenizer.texts_to_sequences([line])[0])

        if len(doc_data) == 0:
            print('File {} has no data'.format(file_list[i]))
        else:
            data.append(doc_data)
        
    return data

def get_embeddings(word_index, w2v, unk_vector):
    embedding_matrix=np.zeros(shape=(len(word_index)+2, w2v.vector_size))  # +2 as keras' tokenizer is 1-based
    missing_words = []
    for word,i in word_index.items():
        if word not in w2v:
            # Try to capitalize it
            if word.capitalize() not in w2v:
                missing_words.append(word)
                embedding_matrix[i] = unk_vector
            else:
                embedding_matrix[i] = w2v[word.capitalize()]
        else:
            embedding_matrix[i] = w2v[word]
    
    # add EOS token
    embedding_matrix[-1] = eos_vector # keras' index the vocab starting from 1
    embedding_matrix[-2] = unk_vector
    return embedding_matrix, missing_words

def get_embedding_matrix(word2embeddings):
    embedding_dim = len(list(word2embeddings.values())[0])
    embedding_matrix = np.zeros(shape=(len(word2embeddings)+2, embedding_dim)) # +2 as keras tokenizer is 1-based
    for i, w in enumerate(word2embeddings): # keras' tokenizer index is 1-based
        embedding_matrix[i+1] = word2embeddings[w]
    
    return embedding_matrix

In [41]:
corpus = read_corpus(file_list)
tokenizer, index_word = build_vocabulary(corpus, num_words=20000, oov_token=unk_token)
embedding_matrix, missing_words = get_embeddings(tokenizer.word_index, w2v, unk_vector)

read_corpus: processing [data/parsed/parsed-12heads.txt]
read_corpus: processing [data/parsed/parsed-1893.txt]
read_corpus: processing [data/parsed/parsed-20160221-thesueno-utf8.txt]
read_corpus: processing [data/parsed/parsed-20160221-thesueno.txt]
read_corpus: processing [data/parsed/parsed-3card-deadmanshill-2016Ap24.txt]
read_corpus: processing [data/parsed/parsed-69krakatoa.txt]
read_corpus: processing [data/parsed/parsed-905-shrapnel.txt]
read_corpus: processing [data/parsed/parsed-abno.txt]
read_corpus: processing [data/parsed/parsed-acg-crossbow.txt]
read_corpus: processing [data/parsed/parsed-acitw.txt]
read_corpus: processing [data/parsed/parsed-actofmurder.txt]
read_corpus: processing [data/parsed/parsed-adverbum.txt]
read_corpus: processing [data/parsed/parsed-afdfr.txt]
read_corpus: processing [data/parsed/parsed-afflicted.txt]
read_corpus: processing [data/parsed/parsed-allthingsdevours.txt]
read_corpus: processing [data/parsed/parsed-aotearoa.txt]
read_corpus: processing

NameError: name 'w2v' is not defined

In [6]:
print('Vocabulary size:', tokenizer.num_words)
print('OOV token index:', tokenizer.word_index[unk_token])
print('EOS token index:', tokenizer.word_index[eos_token])

Vocabulary size: 20002
OOV token index: 20001
EOS token index: 20002


In [7]:
# text = 'Sample sentence with a possible balabiut token and some 1984 plus sentry'
# print(preprocess(text))
# print(prepare_input(text, tokenizer))
vocab_size = len(embedding_matrix)
unk_index = tokenizer.word_index[unk_token]
eos_index = unk_index+1
print('Embedding matrix size:', embedding_matrix.shape)
print('Total triples:', sum([(len(f)-3)//2 for f in data]))
print('Unique words found (UNK, EOS + vocab):', len(tokenizer.word_index))
print('Of which missing words (no embeddings):', len(missing_words))

Embedding matrix size: (20003, 300)
Total triples: 131807
Unique words found (UNK, EOS + vocab): 20002
Of which missing words (no embeddings): 1557


In [9]:
data = prepare_data(corpus, tokenizer)

prepare_data: processing [data/parsed/parsed-12heads.txt]
prepare_data: processing [data/parsed/parsed-1893.txt]
Line 1197 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-20160221-thesueno-utf8.txt]
prepare_data: processing [data/parsed/parsed-20160221-thesueno.txt]
Line 1445 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-3card-deadmanshill-2016Ap24.txt]
prepare_data: processing [data/parsed/parsed-69krakatoa.txt]
prepare_data: processing [data/parsed/parsed-905-shrapnel.txt]
prepare_data: processing [data/parsed/parsed-abno.txt]
Line 1217 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-acg-crossbow.txt]
Line 1825 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-acitw.txt]
prepare_data: processing [data/parsed/parsed-actofmurder.txt]
Line 567 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-adverbum.txt]
Line 

prepare_data: processing [data/parsed/parsed-invisargo.txt]
Line 89 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-jabberwocky.txt]
prepare_data: processing [data/parsed/parsed-jacket4.txt]
prepare_data: processing [data/parsed/parsed-jacqissick.txt]
prepare_data: processing [data/parsed/parsed-jfw.txt]
Line 27 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-ka.txt]
prepare_data: processing [data/parsed/parsed-laidoff-1May2016.txt]
prepare_data: processing [data/parsed/parsed-laidoff-subrosa-1May2016.txt]
Line 539 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-lethe.txt]
prepare_data: processing [data/parsed/parsed-littlebluemen.txt]
prepare_data: processing [data/parsed/parsed-lmwh.txt]
prepare_data: processing [data/parsed/parsed-loose.txt]
Line 675 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-lostpig.txt]
prepare_data: processing [data/

prepare_data: processing [data/parsed/parsed-turkeyspeeds.txt]
Line 1271 is empty. Replacing with "empty line".
Line 1575 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-unclezeb.txt]
prepare_data: processing [data/parsed/parsed-undertow.txt]
prepare_data: processing [data/parsed/parsed-unipool.txt]
prepare_data: processing [data/parsed/parsed-unscientific.txt]
Line 4597 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-vagueness.txt]
prepare_data: processing [data/parsed/parsed-varkana.txt]
prepare_data: processing [data/parsed/parsed-violet.txt]
prepare_data: processing [data/parsed/parsed-wand.txt]
prepare_data: processing [data/parsed/parsed-weapon.txt]
prepare_data: processing [data/parsed/parsed-wedding.txt]
Line 4547 is empty. Replacing with "empty line".
prepare_data: processing [data/parsed/parsed-windjack.txt]
prepare_data: processing [data/parsed/parsed-winterwonderland.txt]
prepare_data: processing [data/

In [46]:
# save various objects for later reuse
with open(os.path.join(DATA_PATH, 'data_20k.pkl'), 'wb') as data_file, open(os.path.join(DATA_PATH, 'params_20k.pkl'), 'wb') as params_file:
    params = {
        'tokenizer': tokenizer,
        'index_word': index_word,
        'W': embedding_matrix,
        'w2e': word2embeddings,
        'missing_words': missing_words
    }
    pkl.dump(data, data_file)
    pkl.dump(params, params_file)
    

In [8]:
def window(seq, n=3, step=1):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s[0],...s[n-1]), (s[0+skip_n],...,s[n-1+skip_n]), ...   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    

    result = result[step:]
    for elem in it:
        result = result + (elem,)
        if len(result) == n:
            yield result
            result = result[step:]

def create_samples(data, test_split=0.1, shuffle=False, max_seq_length=None):    
    samples = []
    for i, play in enumerate(data):
        if max_seq_length is not None:
            chunks = [line[offset:offset+max_seq_length] 
                      for line in play 
                      for offset in range(0, len(line), max_seq_length)]
        else:
            chunks = play
            
        for scene, command, reply in window(chunks, n=3, step=2):
#             if max_seq_length is not None:
#                 sub_scenes  = [scene[offset:offset+max_seq_length]   for offset in range(0, len(scene),   max_seq_length)]
#                 sub_cmds    = [command[offset:offset+max_seq_length] for offset in range(0, len(command), max_seq_length)]
#                 sub_replies = [reply[offset:offset+max_seq_length]   for offset in range(0, len(reply),   max_seq_length)]
                
#                 nb_samples = 
#                 # sample a number of contextual sequences
#                 scenes   = sub_scenes[np.random.choice(range(len(sub_scenes)), len(sub_scenes)//max_seq_length)]
#                 commands = sub_cmds[np.random.choice(range(len(sub_cmds)), len(sub_cmds)//max_seq_length)]
#                 replies   = sub_replies[np.random.choice(range(len(sub_replies)), len(sub_replies)//max_seq_length)]
                
                
#             if len(command) > 10:
#                 command_line = ' '.join([index_word[idx] for idx in command])
#                 print('Found anomalous command for play {} [{}] with length {}: [{}]'.format(
#                     i, os.path.basename(file_list[i]), len(command), command_line))
                
            samples.append((scene, command, reply))
    
    if shuffle:
        np.random.shuffle(samples)
        
    if test_split is not None:
        split = int((1-test_split) * len(samples))
        train_samples = samples[:split]
        test_samples = samples[split:]
        return train_samples, test_samples
    
    return samples

In [83]:
# Define a batch generator
class BatchGenerator(object):
    def __init__(self, data, vocab_size, batch_size=1, reverse_input=False, shuffle=True, max_seq_length=None):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.vocab_size = vocab_size
        self.reverse_input = reverse_input
        self.max_seq_length = max_seq_length
        self.UNK = unk_index
        self.EOS = eos_index
        self.PAD = 0
        
    def generate_batch(self): 
        # every three lines comprise a sample sequence where the first two items
        # are the input and the last one is the output
        i  = 1 # batch counter        
        x_enc = []
        x_dec = []
        y  = []
            
        while True:
            if self.shuffle:
                np.random.shuffle(self.data)
            
            for j, (scene, command, reply) in enumerate(self.data):
                if self.reverse_input:
                    scene = scene[::-1]
                    
                encoder_input  = np.array(scene + command)
                decoder_input  = np.array([self.EOS] + reply)
                decoder_output = np.array(to_categorical(reply + [self.EOS], self.vocab_size))
                    
                x_enc.append(encoder_input)
                x_dec.append(decoder_input)
                y.append(decoder_output)
                
                if i == self.batch_size or j == len(data):
                    if self.batch_size > 1:
                        # pad and return the batch
                        x_enc = sequence.pad_sequences(x_enc, padding='post', value=self.PAD, maxlen=self.max_seq_length)
                        x_dec = sequence.pad_sequences(x_dec, padding='post', value=self.PAD, maxlen=self.max_seq_length)    
                        y     = sequence.pad_sequences(y, padding='post', value=self.PAD, maxlen=self.max_seq_length)

                    x_out, y_out = [np.array(x_enc), np.array(x_dec)], np.array(y)
                    
                    i = 1
                    x_enc = []
                    x_dec = []
                    y = []

                    yield (x_out, y_out)
                else:
                    i += 1 # next sample per batch

In [84]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent
from keras.engine import InputSpec

tfPrint = lambda d, T: tf.Print(input_=T, data=[T, tf.shape(T)], message=d)

def _time_distributed_dense(x, w, b=None, dropout=None,
                            input_dim=None, output_dim=None,
                            timesteps=None, training=None):
    """Apply `y . w + b` for every temporal slice y of x.
    # Arguments
        x: input tensor.
        w: weight matrix.
        b: optional bias vector.
        dropout: wether to apply dropout (same dropout mask
            for every temporal slice of the input).
        input_dim: integer; optional dimensionality of the input.
        output_dim: integer; optional dimensionality of the output.
        timesteps: integer; optional number of timesteps.
        training: training phase tensor or boolean.
    # Returns
        Output tensor.
    """
    if not input_dim:
        input_dim = K.shape(x)[2]
    if not timesteps:
        timesteps = K.shape(x)[1]
    if not output_dim:
        output_dim = K.shape(w)[1]

    if dropout is not None and 0. < dropout < 1.:
        # apply the same dropout pattern at every timestep
        ones = K.ones_like(K.reshape(x[:, 0, :], (-1, input_dim)))
        dropout_matrix = K.dropout(ones, dropout)
        expanded_dropout_matrix = K.repeat(dropout_matrix, timesteps)
        x = K.in_train_phase(x * expanded_dropout_matrix, x, training=training)

    # collapse time dimension and batch dimension together
    x = K.reshape(x, (-1, input_dim))
    x = K.dot(x, w)
    if b is not None:
        x = K.bias_add(x, b)
    # reshape to 3D tensor
    if K.backend() == 'tensorflow':
        x = K.reshape(x, K.stack([-1, timesteps, output_dim]))
        x.set_shape([None, None, output_dim])
    else:
        x = K.reshape(x, (-1, timesteps, output_dim))
    return x

class AttentionDecoder(Recurrent):

    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space

        references:
            Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio.
            "Neural machine translation by jointly learning to align and translate."
            arXiv preprint arXiv:1409.0473 (2014).
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = _time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)
#         self._uxpb = TimeDistributed(Dense(units=self.units, weights=[self.U_a, self.b_a],
#                                              input_dim=self.input_dim)(self.x_seq))

        return super(AttentionDecoder, self).call(x)

    def get_initial_state(self, inputs):
        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1-zt)*stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [98]:
# returns train, inference_encoder and inference_decoder models
def define_models_lstm(src_vocab_size, embedding_matrix, input_shape, dst_vocab_size=None, embedding_dim=300, latent_dim=128, 
                       mask_value=0, trainable_embeddings=False, encoder_depth=1, decoder_depth=1, attention=False):
    # define training encoder. We use return_state to retrieve the hidden states for the encoder and
    # provide them as input to the decoder
    if dst_vocab_size is None:
        dst_vocab_size = src_vocab_size
        
    encoder_inputs = Input(shape=input_shape) # timesteps, features (integer)
    decoder_inputs = Input(shape=input_shape)
    inputs = [encoder_inputs, decoder_inputs]
    
    encoder_masking = Masking(mask_value=mask_value)(encoder_inputs)
    decoder_masking = Masking(mask_value=mask_value)(decoder_inputs)
    
    encoder_embedding = Embedding(input_dim=src_vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], 
                               trainable=trainable_embeddings)(encoder_masking)
    decoder_embedding = Embedding(input_dim=src_vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], 
                               trainable=trainable_embeddings)(decoder_masking)
    
    ######## ENCODER ########
    encoder_lstm = LSTM(latent_dim, return_state=True, return_sequences=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]
    for _ in range(encoder_depth-1):  # DEPTH (the encoder need not be shared, so we can just instantiate a new LSTM object)
        encoder_outputs, state_h, state_c = LSTM(units=latent_dim, return_sequences=True, return_state=True)(encoder_outputs)
        encoder_states = [state_h, state_c]
    
    ######## DECODER ########
    decoder_layers = []  # keep track of deep layers
    
    # define training decoder. It is initialized with the encoder hidden states
    decoder_lstm = LSTM(units=latent_dim, return_sequences=True, return_state=True)        
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    
    if attention:
        decoder_outputs = AttentionDecoder(units=latent_dim, output_dim=embedding_dim)(decoder_outputs)
        
    for _ in range(decoder_depth-1):  # DEPTH
        lstm = LSTM(units=latent_dim, return_sequences=True, return_state=True)
        decoder_layers.append(lstm)
        decoder_outputs, _, _ = lstm(decoder_outputs)
        
    decoder_dense = Dense(dst_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
        
    model = Model(inputs, decoder_outputs)
    
    ####### INFERENCE ENCODER #######
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    
    ####### INFERENCE DECODER #######
    # define inference decoder
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
    
    if attention:
        decoder_outputs = AttentionDecoder(units=latent_dim, output_dim=embedding_dim)(decoder_outputs)
        
    decoder_states = [state_h, state_c]    
    for d in range(decoder_depth - 1):  # DEPTH
        decoder_outputs, state_h, state_c = decoder_layers[d](decoder_outputs)
        decoder_states = [state_h, state_c]
    
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
    # return all models
    return model, encoder_model, decoder_model

In [99]:
def plot_loss(loss, val_loss, color=None, fname=None, legend=False):
        N = len(loss)
        train_loss_plt, = plt.plot(range(0, N), loss)
        val_loss_plt, = plt.plot(range(0, N), val_loss)
        
        if color is not None:
            plt.setp(train_loss_plt, color=color, linestyle='-')
            plt.setp(val_loss_plt, color=color, linestyle='--')
            
        if legend:
            plt.legend((train_loss_plt, val_loss_plt), ('train loss', 'val loss'))
        
        if fname is not None:
            plt.savefig(fname)
        
        return [train_loss_plt, val_loss_plt]

def plot(losses, fname=None):        
    lines = []
    names = []
    colors = [plt.cm.gist_ncar(i) for i in np.linspace(0, 1, len(losses))]
    for i, (loss, val_loss) in enumerate(losses):
        lines.extend(plot_loss(loss, val_loss, colors[i]))
        names.extend(['{} loss'.format(i+1), '{} val loss'.format(i+1)])
    
    plt.legend(lines, names)
    
    if fname is not None:
        plt.savefig(fname)
    else:
        plt.show()

In [100]:
from sklearn.model_selection import train_test_split

class InferenceModelsCheckpoint(Callback):
    def __init__(self, models, filepath, monitor='val_loss', verbose=0):
        self.encoder, self.decoder = models
        self.monitor = monitor
        self.filepath = filepath
        self.verbose = verbose
        self.best = np.Inf
        
    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current < self.best:
            filepath = self.filepath
            if self.verbose > 0:
                print('\nEpoch %05d: saving model to %s' % (epoch + 1, filepath))
                
            self.encoder.save(filepath + '-encinf.h5')
            self.decoder.save(filepath + '-decinf.h5')                
            self.best = current

def train_model(models, train_samples, batch_size, epochs=10, shuffle=True, n_folds=None, train_split=None, 
                model_name=None, max_seq_length=None):
    assert not (n_folds is not None and train_split is not None), ValueError('Either n_folds or train_split should be specified, but not both.')
    assert not (n_folds is None and train_split is None), ValueError('Either n_folds or train_split must be specified.')   
    
    def _run_model(train, val):
        train_generator = BatchGenerator(train, batch_size=batch_size, vocab_size=vocab_size, reverse_input=True, 
                                         shuffle=shuffle, max_seq_length=max_seq_length)
        val_generator = BatchGenerator(val, batch_size=batch_size, vocab_size=vocab_size, reverse_input=True, 
                                       shuffle=shuffle, max_seq_length=max_seq_length)
        
        # utils callbacks
        checkpointer = ModelCheckpoint(filepath=model_name + '.h5', verbose=1, save_best_only=True)
        seq2seq_cp = InferenceModelsCheckpoint(filepath=model_name, verbose=1, models=(encoder, decoder))
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, mode='auto', 
                                      min_delta=0.0001, cooldown=0, min_lr=0)
        early_stop = EarlyStopping(patience=1, min_delta=0.0001, verbose=1)
        callbacks = [checkpointer, seq2seq_cp, reduce_lr, early_stop]
        
        # actual train
        history = model.fit_generator(train_generator.generate_batch(), steps_per_epoch=len(train)//batch_size, epochs=epochs, 
                            validation_data=val_generator.generate_batch(), validation_steps=len(val)//batch_size,
                            callbacks=callbacks)
        
        return history
        
    model, encoder, decoder = models
    train_samples = np.array(train_samples)
    losses = []  # keep track of train and val loss for each fold
    
    if n_folds is None:
        train, val = train_test_split(train_samples, train_size=train_split, shuffle=shuffle)
        
        history = _run_model(train, val)
        # plot current losses
        plot_loss(history.history['loss'], history.history['val_loss'], fname=model_name + '.png')
    else:  
        kfold = KFold(n_folds, shuffle=shuffle)
        for i, (train, val) in enumerate(kfold.split(train_samples)):
            print("Running fold {}/{}".format(i+1, n_folds))

            model_file = model_name + '-fold-{}'.format(i+1)
            history = _run_model(train_samples[train], train_samples[val])

            # record losses for the final plot
            losses.append((history.history['loss'], history.history['val_loss']))

        # plot losses for all folds
        plot(losses, model_name + '.png')

In [101]:
max_seq_length = 200
model_name = 'basic_seq2seq_20k_200_300d_1-1_LSTM'
# create the model
model, encinf, decinf = define_models_lstm(src_vocab_size=vocab_size, input_shape=(max_seq_length,), latent_dim=300, 
                                           embedding_matrix=embedding_matrix, encoder_depth=1, decoder_depth=1, 
                                           trainable_embeddings=False, attention=True)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_55 (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
input_56 (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
masking_39 (Masking)            (None, 200)          0           input_55[0][0]                   
__________________________________________________________________________________________________
masking_40 (Masking)            (None, 200)          0           input_56[0][0]                   
__________________________________________________________________________________________________
embedding_

In [102]:
train_samples, test_samples = create_samples(data, max_seq_length=max_seq_length, test_split=0.01)
print('Train samples:', len(train_samples))
print('Test samples:', len(test_samples))

Train samples: 132060
Test samples: 1334


In [103]:
batch_size = 32
epochs = 5
train_model((model, encinf, decinf), train_samples, batch_size=batch_size, epochs=epochs, 
            train_split=0.95, model_name=model_name, max_seq_length=max_seq_length)

/hpc/sw/python-3.5.2/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Epoch 1/5
 135/3920 [>.............................] - ETA: 3:26:16 - loss: 1.0626 - categorical_accuracy: 0.0047

KeyboardInterrupt: 

In [ ]:
model_name = 'basic_seq2seq_20k_100_300d_1-1_LSTM'
model_file = model_name + '.h5'
# create the model
model, encinf, decinf = define_models_lstm(src_vocab_size=vocab_size, latent_dim=300, embedding_matrix=embedding_matrix,
                             encoder_depth=1, decoder_depth=1, trainable_embeddings=False)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [ ]:
train_samples, test_samples = create_samples(data, max_seq_length=100, test_split=0.01)
print('Train samples:', len(train_samples))
print('Test samples:', len(test_samples))

In [ ]:
batch_size = 128
epochs = 5
train_model((model, encinf, decinf), train_samples, batch_size=batch_size, epochs=epochs, 
            train_split=0.95, model_name=model_name)

In [105]:
max_seq_length = 50
model_name = 'basic_seq2seq_20k_50_300d_1-1_LSTM'
model_file = model_name + '.h5'
# create the model
model, encinf, decinf = define_models_lstm(src_vocab_size=vocab_size, input_shape=(max_seq_length,), latent_dim=300, embedding_matrix=embedding_matrix,
                             encoder_depth=1, decoder_depth=1, trainable_embeddings=False)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [106]:
train_samples, test_samples = create_samples(data, max_seq_length=50, test_split=0.01)
print('Train samples:', len(train_samples))
print('Test samples:', len(test_samples))

Train samples: 157731
Test samples: 1594


In [ ]:
batch_size = 128
epochs = 5
train_model((model, encinf, decinf), train_samples, batch_size=batch_size, epochs=epochs, 
            train_split=0.95, model_name=model_name, max_seq_length=50)

/hpc/sw/python-3.5.2/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Epoch 1/5
1170/1170 [==============================] - 1752s 1s/step - loss: 2.0053 - categorical_accuracy: 0.0594 - val_loss: 1.8167 - val_categorical_accuracy: 0.0740

Epoch 00001: val_loss improved from inf to 1.81666, saving model to basic_seq2seq_20k_50_300d_1-1_LSTM.h5


/home/ruc0029/.local/lib/python3.5/site-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_41 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_40/while/Exit_2:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'lstm_40/while/Exit_3:0' shape=(?, 300) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '



Epoch 00001: saving model to basic_seq2seq_20k_50_300d_1-1_LSTM


/home/ruc0029/.local/lib/python3.5/site-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_41 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'input_61:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'input_62:0' shape=(?, 300) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


Epoch 2/5
1170/1170 [==============================] - 1748s 1s/step - loss: 1.7406 - categorical_accuracy: 0.0823 - val_loss: 1.6766 - val_categorical_accuracy: 0.0887

Epoch 00002: val_loss improved from 1.81666 to 1.67655, saving model to basic_seq2seq_20k_50_300d_1-1_LSTM.h5

Epoch 00002: saving model to basic_seq2seq_20k_50_300d_1-1_LSTM
Epoch 3/5
1170/1170 [==============================] - 1744s 1s/step - loss: 1.6239 - categorical_accuracy: 0.0950 - val_loss: 1.5972 - val_categorical_accuracy: 0.0990

Epoch 00003: val_loss improved from 1.67655 to 1.59717, saving model to basic_seq2seq_20k_50_300d_1-1_LSTM.h5

Epoch 00003: saving model to basic_seq2seq_20k_50_300d_1-1_LSTM
Epoch 4/5
 880/1170 [=====================>........] - ETA: 6:53 - loss: 1.5478 - categorical_accuracy: 0.1032

In [53]:
def prepare_input(input_text, tokenizer):
    return tokenizer.texts_to_sequences([preprocess(input_text)])[0]

def decode_sequence(encinf, decinf, input_seq, vocab_size, max_output_len=50):
    # Encode the input as state vectors.
    states_value = encinf.predict(input_seq)
    print('Sequence encoded')

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))  # time, features
    # Populate the first word with the eos index (.
    target_seq[0, 0] = eos_index
    
    decoder_inputs = [target_seq] + states_value

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    i = 1 # number of sampled words
    while not stop_condition:
        output, h, c = decinf.predict(decoder_inputs)
        sampled_word_index = np.argmax(output[0, -1, :])  # batch, time, features
        sampled_word = index_word[sampled_word_index]

        decoded_sentence += sampled_word + ' '
        # Exit condition: either hit max length or find stop character.
        if sampled_word == eos_token or i > max_output_len:
            stop_condition = True     

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_word_index
        i += 1
        
        # Update states
        decoder_inputs = [target_seq, h, c]

    return decoded_sentence

def prepare_input(input_text, tokenizer):
    return tokenizer.texts_to_sequences([preprocess(input_text)])[0]

def decode_sequence(encinf, decinf, input_seq, vocab_size, max_output_len=50):
    # Encode the input as state vectors.
    states_value = encinf.predict(input_seq)
    print('Sequence encoded')

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))  # time, features
    # Populate the first word with the eos index (.
    target_seq[0, 0] = eos_index
    
    decoder_inputs = [target_seq] + states_value

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    i = 1 # number of sampled words
    while not stop_condition:
        output, h, c = decinf.predict(decoder_inputs)
        sampled_word_index = np.argmax(output[0, -1, :])  # batch, time, features
        sampled_word = index_word[sampled_word_index]

        decoded_sentence += sampled_word + ' '
        # Exit condition: either hit max length or find stop character.
        if sampled_word == eos_token or i > max_output_len:
            stop_condition = True     

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_word_index
        i += 1
        
        # Update states
        decoder_inputs = [target_seq, h, c]

    return decoded_sentence

def beam_decoder_lstm(encinf, decinf, input_seq, k, vocab_size, max_output_len=50):
    # Encode the input as state vectors.
    states_value = encinf.predict(input_seq)
    print('Sequence encoded')

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))  # time, features
    # Populate the first word with the eos index.
    target_seq[0,0] = eos_index

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''    
    decoded_sequences = []
    
    # Init Beam Array
    beams = [(target_seq, states_value, target_seq, 1.0)]
    
    output_len = 1 # number of sampled words
    while not stop_condition:        
        all_candidates = []
        for i in range(len(beams)):  # for each beam, try to append the next predicted word
            # Get the values of the beam
            target_seq, states_value, candidate_seq, score = beams[i]
            
            # predict the next word and create candidate list
            output, h, c = decinf.predict([target_seq] + states_value)
            candidate_score_list = output[0][0]
            
#             argmax = np.argmax(candidate_score_list)
#             print('argmax: {} with value {}'.format(index_word[int(argmax)], candidate_score_list[argmax]))
            
            for j in range(len(candidate_score_list)):  # for each possible word, compute the score               
                # Add each candidate to the target sequence.
                new_candidate_seq = np.append(candidate_seq, [[j]], axis=1)
                candidate_beam = [np.array([[j]]), [h, c],  new_candidate_seq, score*-np.log(candidate_score_list[j])]
                all_candidates.append(candidate_beam)

        # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[-1])
        # select k best
        beams = ordered[:k]
        
        # if a beam has the index of the eos, 
        # 1. Add the beam to decoded_sequences, 
        # 2. Remove it from beams 
        # 3. Lower number of beams (k)
        for bindex, beam in enumerate(beams):
            if beam[0][0][-1] == eos_index:
                print('EOS reached for beam ', bindex)
                decoded_sequences.append(beam)
                del beams[bindex]
                k -= 1
                   
        # Exit condition: either each beam got eos or hit max length
        if len(beams) == 0 or output_len > max_output_len:
            print('Max output length reached')
            # Add the beams that did not have eos so far to the decoded_sequences
            decoded_sequences += beams
            stop_condition = True 

        output_len += 1
        
    decoded_sentences = [(seq[2], seq[-1]) for seq in decoded_sequences]
    decoded_sentences = [(score, [index_word[int(idx)] if idx > 0 else 'NONE' for idx in seq[0]]) for seq, score in decoded_sentences]
    decoded_sentences = [(score, ' '.join(sentence)) for score, sentence in decoded_sentences]

    return decoded_sequences

In [56]:
test_line = """You find yourself in a dimly lit room. Before you you can spot a table. On the other side of the room
a poster hangs askew on the wall. examine poster."""
# test_line = """inventory"""
input_seq = prepare_input(test_line, tokenizer)

In [57]:
# model, encinf, decinf = define_models_lstm(src_vocab_size=vocab_size, latent_dim=300, embedding_matrix=embedding_matrix,
#                              encoder_depth=1, decoder_depth=1, trainable_embeddings=False)
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

# # load the 200-long model
# model_name = 'basic_seq2seq_20k_200_300d_1-1_LSTM'
# if os.path.isfile(model_name + '.h5'):
#     model.load_weights(model_name + '.h5')
#     encinf.load_weights(model_name + '-encinf.h5')
#     decinf.load_weights(model_name + '-decinf.h5')
decode_sequence(encinf, decinf, input_seq, vocab_size, max_output_len=50)

Sequence encoded


'you are in a small clearing in a well marked forest path that extends to the east and south behind the house is a path leading down a path leads northwest and a path leads southeast and a path leads southeast and a passage leads southeast a small passage leads southeast '

In [26]:
beam_decoder_lstm(encinf, decinf, input_seq, 2, vocab_size, max_output_len=50)

Sequence encoded


['UNK chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly sirens sirens sirens sirens chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly sirens sirens sirens sirens chilly chilly chilly chilly chilly',
 'UNK chilly never chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly sirens sirens sirens sirens chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly chilly sirens sirens sirens sirens chilly chilly chilly chilly chilly']